In [8]:
!pip install vecstack
!pip install imbalanced-learn

from vecstack import stacking
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from datetime import datetime
from scipy.stats import randint, uniform
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [9]:
trainfile = r'/content/train.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/content/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe


print(trainData.shape)
print(testData.shape)



(137, 43)
(100000, 42)


In [10]:
trainData.isnull().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
dtype: int64

In [11]:
testData.isnull().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
dtype: int64

In [12]:
trainData.head(5)

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [13]:
testData.head(5)

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,2.0,3.0,0,0,0,0,0,0,0,0
1,1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,...,1.0,3.0,0,0,0,0,0,0,0,0
2,2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,2.0,3.0,0,0,0,0,0,0,0,0
3,3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,2.0,3.0,0,4,0,0,0,0,0,0
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,5.0,3.0,0,0,0,0,0,0,0,0


In [14]:
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id          137 non-null    int64  
 1   Open Date   137 non-null    object 
 2   City        137 non-null    object 
 3   City Group  137 non-null    object 
 4   Type        137 non-null    object 
 5   P1          137 non-null    int64  
 6   P2          137 non-null    float64
 7   P3          137 non-null    float64
 8   P4          137 non-null    float64
 9   P5          137 non-null    int64  
 10  P6          137 non-null    int64  
 11  P7          137 non-null    int64  
 12  P8          137 non-null    int64  
 13  P9          137 non-null    int64  
 14  P10         137 non-null    int64  
 15  P11         137 non-null    int64  
 16  P12         137 non-null    int64  
 17  P13         137 non-null    float64
 18  P14         137 non-null    int64  
 19  P15         137 non-null    i

In [15]:
trainData['City Group'].unique()

array(['Big Cities', 'Other'], dtype=object)

In [16]:
trainData['Type'].unique()

array(['IL', 'FC', 'DT'], dtype=object)

In [17]:
trainData['City Group'] = trainData['City Group'].replace({'Big Cities': 0, 'Other': 1})
trainData['City Group']

0      0
1      0
2      1
3      1
4      1
      ..
132    1
133    0
134    1
135    0
136    0
Name: City Group, Length: 137, dtype: int64

In [18]:
trainData['Type'] = trainData['Type'].replace(to_replace=['IL', 'FC', 'DT'], value=[1, 2, 3])
trainData['Type']

0      1
1      2
2      1
3      1
4      1
      ..
132    2
133    2
134    2
135    2
136    2
Name: Type, Length: 137, dtype: int64

In [19]:
trainData['Open Date'] = pd.to_datetime(trainData['Open Date'])

In [20]:
testData['City Group'].unique()
testData.Type.unique()

testData['Open Date'] = pd.to_datetime(testData['Open Date'])
testData['City Group'] = testData['City Group'].replace({'Big Cities': 0, 'Other': 1})
testData.Type = testData.Type.replace(to_replace=['IL', 'FC', 'DT', 'MB'], value=[1, 2, 3, 4])

In [21]:

trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Id          137 non-null    int64         
 1   Open Date   137 non-null    datetime64[ns]
 2   City        137 non-null    object        
 3   City Group  137 non-null    int64         
 4   Type        137 non-null    int64         
 5   P1          137 non-null    int64         
 6   P2          137 non-null    float64       
 7   P3          137 non-null    float64       
 8   P4          137 non-null    float64       
 9   P5          137 non-null    int64         
 10  P6          137 non-null    int64         
 11  P7          137 non-null    int64         
 12  P8          137 non-null    int64         
 13  P9          137 non-null    int64         
 14  P10         137 non-null    int64         
 15  P11         137 non-null    int64         
 16  P12         137 non-null  

In [22]:
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Id          100000 non-null  int64         
 1   Open Date   100000 non-null  datetime64[ns]
 2   City        100000 non-null  object        
 3   City Group  100000 non-null  int64         
 4   Type        100000 non-null  int64         
 5   P1          100000 non-null  int64         
 6   P2          100000 non-null  float64       
 7   P3          100000 non-null  float64       
 8   P4          100000 non-null  float64       
 9   P5          100000 non-null  int64         
 10  P6          100000 non-null  int64         
 11  P7          100000 non-null  int64         
 12  P8          100000 non-null  int64         
 13  P9          100000 non-null  int64         
 14  P10         100000 non-null  int64         
 15  P11         100000 non-null  int64         
 16  P12

In [23]:
TrainCols = list(trainData.columns.values)
TestCols = list(testData.columns.values)
print(TrainCols)
print(TestCols)

['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'revenue']
['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37']


In [24]:
testDataOrignal = testData
testDataOrignal.shape

(100000, 42)

In [25]:
train_df, test_df = train_test_split(trainData, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(109, 28)

In [26]:
train_df = train_df.dropna()
test_df = test_df.dropna()

In [27]:
train_df.columns

Index(['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4',
       'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15',
       'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25',
       'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35',
       'P36', 'P37', 'revenue'],
      dtype='object')

In [28]:
input = ['P1', 'P2', 'P3', 'P4',
       'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15',
       'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25',
       'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35',
       'P36', 'P37']

target = 'revenue'

In [29]:
train_inputs = train_df[input]
train_targets = train_df[target]

test_inputs = test_df[input]
test_targets = test_df[target]


In [30]:
testDataOrignal = testDataOrignal[input]
testDataOrignal

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,1,4.0,4.0,4.0,1,2,5,4,5,5,...,2.0,3.0,0,0,0,0,0,0,0,0
1,3,4.0,4.0,4.0,2,2,5,3,4,4,...,1.0,3.0,0,0,0,0,0,0,0,0
2,3,4.0,4.0,4.0,2,2,5,4,4,5,...,2.0,3.0,0,0,0,0,0,0,0,0
3,2,4.0,4.0,4.0,2,3,5,4,5,4,...,2.0,3.0,0,4,0,0,0,0,0,0
4,2,4.0,4.0,4.0,1,2,5,4,5,4,...,5.0,3.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5,5.0,4.0,4.0,2,2,5,5,4,4,...,2.0,3.0,0,0,0,0,0,0,0,0
99996,1,2.0,4.0,3.0,1,1,1,5,5,5,...,1.0,3.0,5,0,0,0,4,0,0,0
99997,4,5.0,4.0,4.0,1,2,5,3,4,5,...,4.0,1.0,5,5,3,2,4,4,4,2
99998,12,7.5,6.0,6.0,4,4,10,10,10,10,...,7.5,5.0,0,0,0,4,0,0,0,0


In [56]:
#Decision Tree Regressor ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY
clf = DecisionTreeRegressor()
clf.fit(train_inputs, train_targets )
clf_predict_Train=clf.predict(train_inputs)

#clf.feature_importances_
mean_squared_error(train_targets,clf_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(train_targets,clf_predict_Train)))
clf_predict_Test=clf.predict(test_inputs)
mean_squared_error(test_targets,clf_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(test_targets,clf_predict_Test)))


RMSE (training) for Decision Tree:  0.000000
RMSE (Test Data) for Decision Tree:10152663003577.892578


In [32]:
#Randomized Search for hyperparameter tuning
search_grid={
    'max_features': [ .5, .6, .7],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 20, 30, 40, 50],
    'min_impurity_decrease': uniform(0, 0.5),
             }
model_random = RandomizedSearchCV(clf,search_grid,n_iter=15)
model_random.fit(train_inputs, train_targets)
grid_parm_model=model_random.best_params_
print(grid_parm_model)

{'max_depth': 10, 'max_features': None, 'max_leaf_nodes': 30, 'min_impurity_decrease': 0.19981486693091127, 'splitter': 'random'}


In [65]:
hype_clf = DecisionTreeRegressor(max_depth= 30, max_features= 'sqrt', max_leaf_nodes= 40, min_impurity_decrease= 0.38191795609362544, splitter= 'best')
hype_clf.fit(train_inputs, train_targets)
hype_clf_predict_Train=hype_clf.predict(train_inputs)

#clf.feature_importances_
mean_squared_error(train_targets,hype_clf_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(train_targets,clf_predict_Train)))
hype_clf_predict_Test=hype_clf.predict(test_inputs)
mean_squared_error(test_targets,hype_clf_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(test_targets,hype_clf_predict_Test)))
# evaluate(hype_clf)

RMSE (training) for Decision Tree:  0.000000
RMSE (Test Data) for Decision Tree:11473071846514.785156


In [34]:
#Save predictions
df_DT=pd.DataFrame()
df_DT['revenue']=hype_clf_predict_Test
df_DT['revenue']
export_csv = df_DT.to_csv(r'/gdrive/My Drive/ML/Assignment4/DT_Model.csv')


In [35]:
#Random Forest Regressor==============================================================================
#=================================================================================================

rfc = RandomForestRegressor()
rfc.fit(train_inputs, train_targets)
rfc_predict_Train=rfc.predict(train_inputs)

mean_squared_error(train_targets, rfc_predict_Train)
print("RMSE (training) for Random Forest:{0:10f}".format(mean_squared_error(train_targets,rfc_predict_Train)))
rfc_predict_Test=rfc.predict(test_inputs)
mean_squared_error(test_targets,rfc_predict_Test)
print("RMSE (Test Data) for Random Forest:{0:10f}".format(mean_squared_error(test_targets,rfc_predict_Test)))



RMSE (training) for Random Forest:796765214518.076294
RMSE (Test Data) for Random Forest:11000833245131.837891


In [36]:
#Randomized Search for hyperparameter tuning
search_grid={'max_depth': [ 65, 75, 100],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]
          }
model_rfc = RandomizedSearchCV(rfc,search_grid,n_iter=15)
model_rfc.fit(train_inputs, train_targets)
grid_model=model_rfc.best_params_
print(grid_model)

{'max_features': 'log2', 'max_depth': 75, 'bootstrap': True}


In [37]:
hype_rfc = RandomForestRegressor(max_features= 'sqrt',  max_depth = 65, bootstrap= True)
hype_rfc.fit(train_inputs, train_targets)
hype_rfc_predict_Train=hype_rfc.predict(train_inputs)

#clf.feature_importances_
mean_squared_error(train_targets,hype_rfc_predict_Train)
print("RMSE (training) for Random Forest:{0:10f}".format(mean_squared_error(train_targets,rfc_predict_Train)))
hype_rfc_predict_Test=hype_rfc.predict(test_inputs)
mean_squared_error(test_targets,hype_rfc_predict_Test)
print("RMSE (Test Data) for Randon Forest:{0:10f}".format(mean_squared_error(test_targets,hype_rfc_predict_Test)))
# evaluate(hype_clf)

RMSE (training) for Random Forest:796765214518.076294
RMSE (Test Data) for Randon Forest:10927301412900.962891


In [38]:
#Save predictions
df_rfc=pd.DataFrame()
df_rfc['TARGET']=rfc_predict_Test
export_csv = df_rfc.to_csv(r'/gdrive/My Drive/ML/Assignment4/RF_Model.csv')

In [39]:
#Gradient Boosting Regressor================================================================================

abc =GradientBoostingRegressor()
abc.fit(train_inputs, train_targets)
abc_predict_Train=abc.predict(train_inputs)

mean_squared_error(train_targets ,abc_predict_Train)
print("RMSE (training) for Gradient Boosting:{0:10f}".format(mean_squared_error(train_targets,abc_predict_Train)))
abc_predict_Test=rfc.predict(test_inputs)
mean_squared_error(test_targets,abc_predict_Test)
print("RMSE (Test Data) for Gradient Boosting:{0:10f}".format(mean_squared_error(test_targets ,abc_predict_Test)))




RMSE (training) for Gradient Boosting:228724015060.073517
RMSE (Test Data) for Gradient Boosting:11000833245131.837891


In [40]:
#Randomized Search for hyperparameter tuning
search_grid={
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.1),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.5, 1.0),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2', None]
}
model_abc = RandomizedSearchCV(abc,search_grid,n_iter=15)
model_abc.fit(train_inputs, train_targets)
grid_model_abc=model_abc.best_params_
print(grid_model_abc)

{'learning_rate': 0.01397422929983342, 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 19, 'n_estimators': 151, 'subsample': 0.5834266017726071}


In [41]:
hype_abc = GradientBoostingRegressor(learning_rate= 0.013860725741271677, max_depth= 5, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split= 16, n_estimators= 92, subsample = 0.5504873378467406)
hype_abc.fit(train_inputs, train_targets)
hype_abc_predict_Train=hype_abc.predict(train_inputs)

#clf.feature_importances_
mean_squared_error(train_targets,hype_abc_predict_Train)
print("RMSE (training) for Gradient Boosting:{0:10f}".format(mean_squared_error(train_targets,abc_predict_Train)))
hype_abc_predict_Test=hype_abc.predict(test_inputs)
mean_squared_error(test_targets,hype_abc_predict_Test)
print("RMSE (Test Data) for Gradient Boosting:{0:10f}".format(mean_squared_error(test_targets,hype_abc_predict_Test)))

RMSE (training) for Gradient Boosting:228724015060.073517
RMSE (Test Data) for Gradient Boosting:12074452842051.125000


In [42]:
#Save predictions
df_abc=pd.DataFrame()
df_abc['revenue']=abc_predict_Test
export_csv = df_abc.to_csv(r'/gdrive/My Drive/ML/Assignment4/GB_Model.csv')

In [43]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(train_inputs, train_targets)

In [44]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingRegressor(), RandomForestRegressor(), DecisionTreeRegressor() ]

S_Train, S_Test = stacking(models,
                           train_inputs, train_targets, test_inputs,
                           regression=True,

                           mode='oof_pred_bag',

                           needs_proba=False,

                           save_dir=None,

                           n_folds=4,

                           verbose=2,
                           shuffle=True)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingRegressor]
    fold  0:  [1694140.21756257]
    fold  1:  [1286092.51586381]
    fold  2:  [2018160.49856421]
    fold  3:  [1682465.99344548]
    ----
    MEAN:     [1670214.80635902] + [259489.23660998]
    FULL:     [1670434.30554437]

model  1:     [RandomForestRegressor]
    fold  0:  [1580944.00142857]
    fold  1:  [1263322.57740741]
    fold  2:  [1906905.20851852]
    fold  3:  [1621145.69296296]
    ----
    MEAN:     [1593079.37007937] + [228123.42319336]
    FULL:     [1592968.03642202]

model  2:     [DecisionTreeRegressor]
    fold  0:  [2322422.28571429]
    fold  1:  [1918084.37037037]
    fold  2:  [2661161.33333333]
    fold  3:  [1998278.40740

In [45]:
model = RandomForestRegressor()

model = model.fit(train_inputs, train_targets)

#Hyperparameter Tuning & K-fold Cross Validation
# Doing HyperParameter Tuning & K-fold Cross Validation on RandomForestRegressor
parameters={
              'min_samples_leaf' : range(10,100,10),
              'n_estimators':[20,30,40],
              'max_depth': [65, 75, 100],
              'max_features': ['auto', 'sqrt', 'log2', None],
              'bootstrap': [True, False]
            }

# After doing Hyperparameter Tuning,Using the RandomForestRegressor with the best parameters
rf_random = RandomizedSearchCV(model,parameters,n_iter=25,cv=5)
rf_random.fit(train_inputs, train_targets)
grid_parm=rf_random.best_params_
print('The Best Paramteres for Deceision Tree Regressor is : ',grid_parm)
print('\nThe Best Score for Decision Tree Regressor is : ',rf_random.best_score_)

The Best Paramteres for Deceision Tree Regressor is :  {'n_estimators': 30, 'min_samples_leaf': 40, 'max_features': 'sqrt', 'max_depth': 100, 'bootstrap': False}

The Best Score for Decision Tree Regressor is :  -0.013630738183341129


In [51]:
y_pred_train = rf_random.predict(train_inputs)
y_pred_test = rf_random.predict(testDataOrignal)
res_df=pd.DataFrame()
res_df['prediction']=y_pred_test
res_df['prediction']

0        4.282385e+06
1        4.166602e+06
2        4.393463e+06
3        4.189034e+06
4        4.353474e+06
             ...     
99995    4.399526e+06
99996    3.994075e+06
99997    4.760985e+06
99998    4.836969e+06
99999    4.380741e+06
Name: prediction, Length: 100000, dtype: float64

In [61]:
id = testData['Id']
stack_res = pd.concat([id,res_df['prediction']],axis=1)

In [63]:
stack_res.to_csv(r'/gdrive/My Drive/ML/Assignment4/Stacking_results_Final.csv',index=None)

In [46]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
#model = GradientBoostingRegressor()
#model = model.fit(S_Train, train_targets)
#y_pred_train = model.predict(S_Train)
#y_pred_test = model.predict(S_Test)

In [57]:
#mean_squared_error(train_targets,y_pred_train)
#print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(train_targets,y_pred_train)))
#mean_squared_error(test_targets,y_pred_test)
#print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(test_targets,y_pred_test)))

In [58]:
#Randomized Search for hyperparameter tuning
#search_grid={
#    'n_estimators': randint(50, 200),
#    'learning_rate': uniform(0.01, 0.1),
#    'max_depth': randint(3, 10),
#    'subsample': uniform(0.5, 1.0),
#    'min_samples_split': randint(2, 20),
#    'min_samples_leaf': randint(1, 20),
#    'max_features': ['auto', 'sqrt', 'log2', None]
#}
#model_stacked = RandomizedSearchCV(model,search_grid,n_iter=15)
#model_stacked.fit(S_Train, train_targets)
#grid_model_stacked=model_stacked.best_params_
#print(grid_model_stacked)

In [59]:
#hype_model = GradientBoostingRegressor(learning_rate= 0.025150568447163617, max_depth= 4, max_features= 'auto', min_samples_leaf= 10, min_samples_split= 10, n_estimators= 114)
#hype_model.fit(S_Train, train_targets)
#hype_model_predict_Train= hype_model.predict(train_inputs)

#clf.feature_importances_
#mean_squared_error(train_targets ,hype_model_predict_Train)
#print("RMSE (training) for Random Forest:{0:10f}".format(mean_squared_error(S_Train,hype_model_predict_Train)))
#hype_model_predict_Test=hype_model.predict(test_inputs)
#mean_squared_error(test_targets,hype_model_predict_Test)
#print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(S_Train,hype_model_predict_Test)))


In [162]:
#Save predictions
#df_stacking=pd.DataFrame()
#df_stacking['revenue']=
#export_csv = df_rfc.to_csv(r'/gdrive/My Drive/ML/Assignment4/Stacking_results.csv')